In [15]:
import numpy as np
import pandas as pd
# import sklearn as sk
import plotly.graph_objects as go
import plotly.express as px


In [16]:
import sys
sys.path.append('C:\\Users\\filip\meu_computador\\Github\\repositories\\kaggle\\kaggle_challenges\\doordash_challenge')
from doordash_challenge.models.functions.Cleaner import DataCleaner
from doordash_challenge.models.functions.Handler import DataHandler

# Reading data

In [33]:
whole_data = pd.read_csv('C:\\Users\\filip\\meu_computador\\Github\\repositories\\kaggle\\kaggle_challenges\\doordash_challenge\\data\\historical_data.csv')
whole_data['created_at'] = pd.to_datetime(whole_data['created_at'])
whole_data['actual_delivery_time'] = pd.to_datetime(whole_data['actual_delivery_time'])
whole_data  = whole_data.sort_values('created_at')
whole_data = whole_data.drop(['min_item_price', 'max_item_price'],axis=1)

## Simple Cleaning

In [34]:
data_clean_temporal = DataCleaner.add_temporal_variables(whole_data)
data_clean_target = DataCleaner.add_target_variables(data_clean_temporal.copy(), delivery_threshold=3)
data_cleaning = DataCleaner.remove_negative_values(data_clean_target.copy())


In [35]:
# # filling categorinal nan values 
# whole_data['market_id'] = whole_data['market_id'].fillna('not informed')
# whole_data['market_id'] = whole_data['market_id'].astype(str)

# whole_data['order_protocol'] = whole_data['order_protocol'].fillna('not informed')
# whole_data['order_protocol'] = whole_data['order_protocol'].astype(str)

# whole_data['store_primary_category'] = whole_data['store_primary_category'].fillna('not informed')


## Input missing data of categorical columns

In [ ]:
fillna_datahandler = DataHandler(data=data_cleaning.copy(),column_methods={'market_id':'fill with new category', 'store_primary_category':'fill with new category'})
data_cleaning_no_categorical_na =  fillna_datahandler.fill_na_with_new_category()

In [50]:
data_cleaning_no_categorical_na.shape

(197202, 21)

## Correction of the whole data for EDA

In [63]:
curated_data = DataCleaner.clean_conflict_category(data_cleaning_no_categorical_na.copy())

In [82]:
data_cleaning_no_categorical_na.loc[data_cleaning_no_categorical_na['store_id'].eq(4210)]['market_id'].value_counts()

market_id
2.0             474
4.0               4
1.0               4
3.0               2
5.0               2
not informed      2
6.0               2
Name: count, dtype: int64

In [83]:
curated_data.loc[curated_data['store_id'].eq(4210)]['market_id'].value_counts()

market_id
2.0    490
Name: count, dtype: int64

## Splitting training and testing to avoid data leeakge 

In [58]:
data_cleaning_no_categorical_na = data_cleaning_no_categorical_na.sort_values('created_at')
training_split = int(np.ceil(0.8*data_cleaning_no_categorical_na.shape[0]))
training_data = data_cleaning_no_categorical_na[:training_split] 
testing_data = data_cleaning_no_categorical_na[training_split:]

In [73]:
curated_training_data = DataCleaner.clean_conflict_category(training_data.copy())

In [84]:
training_data.loc[training_data['store_id'].eq(4210)]['market_id'].value_counts()

market_id
2.0             370
1.0               4
4.0               3
5.0               2
not informed      2
6.0               2
3.0               1
Name: count, dtype: int64

In [85]:
curated_training_data.loc[curated_training_data['store_id'].eq(4210)]['market_id'].value_counts()

market_id
2.0    384
Name: count, dtype: int64

## Input missing data

In [20]:
whole_data['time_of_day'].value_counts(normalize=True)*100

time_of_day
Afternoon    65.814373
Evening      19.052009
Night        14.409810
Morning       0.723808
Name: proportion, dtype: float64

## Correct food_category and market_id based on store

In [21]:
whole_data_market_id = whole_data\
        .groupby(['market_id','store_id'],as_index=False)\
        .agg(first_occurence=('created_at','min'), count_subtotal=('subtotal','count'), avg_subtotal=('subtotal','mean'), median_subtotal=('subtotal','median'))\
        .sort_values(['store_id','count_subtotal'],ascending=[False, False])

whole_data_primary_category = whole_data\
        .groupby(['store_primary_category','store_id'],as_index=False)\
        .agg(first_occurence=('created_at','min'), count_subtotal=('subtotal','count'), avg_subtotal=('subtotal','mean'), median_subtotal=('subtotal','median'))\
        .sort_values(['store_id', 'count_subtotal'],ascending=[False, False])

In [22]:
display(whole_data_market_id.loc[whole_data_market_id['store_id'].eq(314)])
display(whole_data_primary_category.loc[whole_data_primary_category['store_id'].eq(314)])


,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
3009,2.0,314,2015-01-21 17:58:11,787,1797.362135,1549.0
145,1.0,314,2015-01-25 17:33:13,9,2529.777778,2123.0
7505,4.0,314,2015-01-22 21:03:31,8,1763.500000,1855.0
4966,3.0,314,2015-01-25 17:21:43,4,2331.000000,1223.0
11994,not informed,314,2015-01-25 02:31:33,4,1253.250000,1248.0
9311,5.0,314,2015-01-28 18:33:26,2,1972.000000,1972.0
10706,6.0,314,2015-02-14 05:59:25,1,1074.000000,1074.0


,store_primary_category,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
8751,mexican,314,2015-01-21 17:58:11,776,1784.413660,1548.0
10030,not informed,314,2015-01-27 04:09:38,6,3103.166667,1545.0
225,american,314,2015-01-23 05:42:34,5,2964.800000,2348.0
11539,pizza,314,2015-01-28 18:33:26,3,1940.666667,1348.0
4395,dessert,314,2015-02-01 20:53:15,2,2316.500000,2316.5
6220,indian,314,2015-01-22 21:03:31,2,1161.000000,1161.0
7364,japanese,314,2015-02-10 02:09:34,2,1705.500000,1705.5
12864,sandwich,314,2015-02-10 21:34:03,2,861.500000,861.5
14348,thai,314,2015-01-24 02:03:50,2,2214.500000,2214.5
15021,vietnamese,314,2015-02-06 19:39:19,2,2160.000000,2160.0


In [ ]:


whole_data_market_id_maxs = whole_data_market_id.groupby('store_id', as_index=False).agg(count_subtotal=('count_subtotal','max')) 
whole_data_market_id_mins = whole_data_market_id.groupby('store_id', as_index=False).agg(first_occurence=('first_occurence','min')) 
whole_data_market_id_filtered = whole_data_market_id.merge(whole_data_market_id_maxs, how='inner', on=['store_id','count_subtotal'])
whole_data_market_id_unique = whole_data_market_id.merge(whole_data_market_id_mins, how='inner', on=['store_id','first_occurence'])


whole_data_primary_category_maxs = whole_data_primary_category.groupby('store_id', as_index=False).agg(count_subtotal=('count_subtotal','max')) 
whole_data_primary_category_mins = whole_data_primary_category.groupby('store_id', as_index=False).agg(first_occurence=('first_occurence','min')) 
whole_data_primary_category_filtered = whole_data_primary_category.merge(whole_data_primary_category_maxs, how='inner', on=['store_id','count_subtotal'])
whole_data_primary_category_unique = whole_data_primary_category.merge(whole_data_primary_category_mins, how='inner', on=['store_id','first_occurence'])



In [24]:
display(whole_data_market_id_filtered.loc[whole_data_market_id_filtered['store_id'].eq(314)])
display(whole_data_primary_category_filtered.loc[whole_data_primary_category_filtered['store_id'].eq(314)])

,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
6491,2.0,314,2015-01-21 17:58:11,787,1797.362135,1549.0


,store_primary_category,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
6493,mexican,314,2015-01-21 17:58:11,776,1784.41366,1548.0


In [25]:
whole_data.shape

(197428, 21)

In [26]:
whole_data_market_id_unique

,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
0,3.0,6987,2015-01-25 04:58:46,2,4350.000000,4350.0
1,4.0,6986,2015-01-21 20:25:49,36,1179.888889,998.0
2,1.0,6985,2015-01-31 03:45:03,2,1085.000000,1085.0
3,6.0,6984,2015-01-31 02:49:56,4,3087.500000,3340.0
4,1.0,6983,2015-01-21 22:01:56,24,2472.000000,2451.5
...,...,...,...,...,...,...
6738,6.0,5,2015-01-26 18:29:58,1,2256.000000,2256.0
6739,1.0,4,2015-01-22 02:26:28,130,1592.046154,1303.0
6740,3.0,3,2015-02-05 03:58:47,3,4133.333333,4200.0
6741,1.0,2,2015-01-22 20:40:45,5,3764.400000,3420.0


In [27]:
whole_data_test = whole_data.drop(['market_id', 'store_primary_category'],axis=1)
whole_data_test = \
        whole_data_test\
            .merge(whole_data_primary_category_unique[['store_id','store_primary_category']], on='store_id')\
            .merge(whole_data_market_id_unique[['store_id','market_id']], on='store_id')

In [28]:
print(whole_data.shape)
print(whole_data_test.shape)

(197428, 21)
(197428, 21)


In [29]:
whole_data.loc[whole_data['store_id'].eq(791)]['store_primary_category'].value_counts()

store_primary_category
not informed    180
asian             1
sushi             1
thai              1
greek             1
Name: count, dtype: int64

In [30]:
whole_data_test.loc[whole_data_test['store_id'].eq(791)]

,created_at,actual_delivery_time,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,...,estimated_store_to_consumer_driving_duration,weekday,weekend,hour,time_of_day,estimated_delivery_time,delivery_time,delivery_time_hours,store_primary_category,market_id
85340,2015-01-21 21:48:26,2015-01-21 22:13:37,791,1.0,2,2298,2,29.0,21.0,22.0,...,622.0,2,0,21,Night,1068.0,1511.0,0.419722,not informed,4.0
85341,2015-01-22 00:30:23,2015-01-22 00:55:38,791,1.0,1,999,1,41.0,22.0,26.0,...,705.0,3,0,0,Afternoon,1151.0,1515.0,0.420833,not informed,4.0
85342,2015-01-22 00:43:15,2015-01-22 01:06:36,791,1.0,3,1797,3,43.0,29.0,31.0,...,167.0,3,0,0,Afternoon,613.0,1401.0,0.389167,not informed,4.0
85343,2015-01-22 01:29:24,2015-01-22 02:13:56,791,1.0,6,4594,6,73.0,71.0,97.0,...,479.0,3,0,1,Afternoon,925.0,2672.0,0.742222,not informed,4.0
85344,2015-01-22 02:57:00,2015-01-22 03:49:13,791,1.0,1,1099,1,96.0,78.0,121.0,...,440.0,3,0,2,Afternoon,886.0,3133.0,0.870278,not informed,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85519,2015-02-17 23:10:04,2015-02-17 23:58:55,791,1.0,3,2597,3,8.0,8.0,11.0,...,815.0,1,0,23,Night,1261.0,2931.0,0.814167,not informed,4.0
85520,2015-02-18 01:48:02,2015-02-18 02:22:35,791,1.0,4,2896,4,73.0,71.0,114.0,...,802.0,2,0,1,Afternoon,1248.0,2073.0,0.575833,not informed,4.0
85521,2015-02-18 02:36:50,2015-02-18 03:22:34,791,5.0,2,2350,2,94.0,90.0,153.0,...,752.0,2,0,2,Afternoon,1198.0,2744.0,0.762222,not informed,4.0
85522,2015-02-18 03:25:30,2015-02-18 03:57:00,791,1.0,1,1199,1,90.0,86.0,150.0,...,619.0,2,0,3,Afternoon,1065.0,1890.0,0.525000,not informed,4.0


In [31]:
display(whole_data_primary_category_filtered.loc[whole_data_primary_category_filtered['store_id'].eq(2387)])
whole_data_primary_category_unique.loc[whole_data_primary_category_unique['store_id'].eq(2387)]

,store_primary_category,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
4486,burger,2387,2015-01-22 02:47:51,1,3690.0,3690.0
4487,korean,2387,2015-02-07 23:36:57,1,2790.0,2790.0
4488,mediterranean,2387,2015-01-25 03:44:16,1,1395.0,1395.0


,store_primary_category,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
4445,burger,2387,2015-01-22 02:47:51,1,3690.0,3690.0


In [32]:
display(whole_data_market_id_filtered.loc[whole_data_market_id_filtered['store_id'].eq(104)])
whole_data_market_id_unique.loc[whole_data_market_id_unique['store_id'].eq(104)]

,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
6695,4.0,104,2015-01-28 19:23:16,1,2700.0,2700.0
6696,5.0,104,2015-02-11 23:22:02,1,1375.0,1375.0
6697,6.0,104,2015-02-06 23:08:21,1,1375.0,1375.0


,market_id,store_id,first_occurence,count_subtotal,avg_subtotal,median_subtotal
6643,4.0,104,2015-01-28 19:23:16,1,2700.0,2700.0


In [33]:
whole_data_test.loc[whole_data_test['order_protocol'].isna()]

,created_at,actual_delivery_time,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,...,estimated_store_to_consumer_driving_duration,weekday,weekend,hour,time_of_day,estimated_delivery_time,delivery_time,delivery_time_hours,store_primary_category,market_id


In [34]:
whole_data_test.head()

,created_at,actual_delivery_time,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,...,estimated_store_to_consumer_driving_duration,weekday,weekend,hour,time_of_day,estimated_delivery_time,delivery_time,delivery_time_hours,store_primary_category,market_id
0,2014-10-19 05:24:15,2015-01-25 19:11:54,3560,1.0,1,1695,1,NaN,NaN,NaN,...,412.0,6,1,5,Afternoon,858.0,8516859.0,2365.794167,italian,1.0
1,2015-01-22 03:17:40,2015-01-22 03:54:40,3560,3.0,1,1595,1,51.0,44.0,45.0,...,786.0,3,0,3,Afternoon,1232.0,2220.0,0.616667,italian,1.0
2,2015-01-22 19:18:53,2015-01-22 20:10:54,3560,1.0,4,2480,3,25.0,25.0,25.0,...,376.0,3,0,19,Evening,822.0,3121.0,0.866944,italian,1.0
3,2015-01-24 02:17:39,2015-01-24 03:23:21,3560,1.0,1,1795,1,72.0,73.0,88.0,...,1023.0,5,1,2,Afternoon,1469.0,3942.0,1.095000,italian,1.0
4,2015-01-25 02:26:34,2015-01-25 03:21:42,3560,1.0,5,3015,4,69.0,63.0,93.0,...,1019.0,6,1,2,Afternoon,1465.0,3308.0,0.918889,italian,1.0


## Input missing data

In [35]:
x = whole_data[['store_id','store_primary_category']].value_counts().sort_index().to_frame().reset_index()
x['store_id'].value_counts().head()

store_id
6865    26
314     23
1311    23
3937    22
1686    22
Name: count, dtype: int64

In [36]:
y = whole_data.loc[whole_data['store_id'].isin([314]),['store_id','market_id','store_primary_category']]
y = y.drop_duplicates()
y.sort_values(['market_id','store_id']).head(50)

,store_id,market_id,store_primary_category
158434,314,1.0,mexican
158809,314,1.0,gastropub
158245,314,1.0,seafood
158215,314,1.0,american
158579,314,1.0,indian
158228,314,1.0,japanese
158417,314,1.0,vietnamese
158550,314,1.0,mediterranean
158305,314,2.0,mexican
158232,314,2.0,sushi


In [37]:
whole_data.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_onshift_dashers','median'),).pivot(index='market_id',columns='time_of_day', values='median_value')

time_of_day,Afternoon,Evening,Morning,Night
market_id,,,,
1.0,24.0,14.0,6.0,14.0
2.0,75.0,39.0,18.0,33.0
3.0,19.0,10.0,5.0,11.0
4.0,72.0,47.0,11.0,30.0
5.0,24.0,15.0,9.0,14.0
6.0,47.0,25.0,12.0,21.0
not informed,47.5,24.0,6.5,21.0


In [38]:
whole_data.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_onshift_dashers','median'),).pivot(index='market_id',columns='time_of_day', values='median_value')

time_of_day,Afternoon,Evening,Morning,Night
market_id,,,,
1.0,24.0,14.0,6.0,14.0
2.0,75.0,39.0,18.0,33.0
3.0,19.0,10.0,5.0,11.0
4.0,72.0,47.0,11.0,30.0
5.0,24.0,15.0,9.0,14.0
6.0,47.0,25.0,12.0,21.0
not informed,47.5,24.0,6.5,21.0


In [370]:
x = whole_data_test[['store_id','store_primary_category']].value_counts().sort_index().to_frame().reset_index()
x['store_id'].value_counts().head()

store_id
1       1
4645    1
4671    1
4670    1
4668    1
Name: count, dtype: int64

In [371]:
y = whole_data_test.loc[whole_data_test['store_id'].isin([314]),['store_id','market_id','store_primary_category']]
y = y.drop_duplicates()
y.sort_values(['market_id','store_id']).head(50)

,store_id,market_id,store_primary_category
10271,314,2.0,mexican


In [378]:
whole_data_test

,created_at,actual_delivery_time,store_id,order_protocol,total_items,subtotal,num_distinct_items,total_onshift_dashers,total_outstanding_orders,estimated_order_place_duration,...,estimated_delivery_time,delivery_time,delivery_time_hours,weekday,hour,time_of_day,weekend,availabe_dashers_onshift,store_primary_category,market_id
0,2015-01-21 15:22:03,2015-01-21 16:17:43,2966,4.0,2,1058,2,2.0,1.0,251,...,714.0,3340.0,0.927778,2,15,Afternoon,0,1.0,dessert,1.0
1,2015-01-22 18:13:58,2015-01-22 18:42:44,2966,4.0,2,1598,1,10.0,8.0,251,...,996.0,1726.0,0.479444,3,18,Evening,0,2.0,dessert,1.0
2,2015-01-22 18:57:55,2015-01-22 19:19:14,2966,4.0,1,599,1,17.0,12.0,251,...,959.0,1279.0,0.355278,3,18,Evening,0,5.0,dessert,1.0
3,2015-01-22 19:22:16,2015-01-22 19:55:00,2966,4.0,2,198,1,19.0,17.0,251,...,963.0,1964.0,0.545556,3,19,Evening,0,2.0,dessert,1.0
4,2015-01-24 00:46:42,2015-01-24 01:28:12,2966,4.0,1,1048,1,24.0,18.0,251,...,679.0,2490.0,0.691667,5,0,Afternoon,1,5.0,dessert,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197197,2015-02-18 02:22:11,2015-02-18 03:06:27,482,1.0,2,2295,2,9.0,6.0,446,...,795.0,2656.0,0.737778,2,2,Afternoon,0,3.0,japanese,1.0
197198,2015-02-18 02:37:52,2015-02-18 04:24:14,2640,1.0,2,2250,2,109.0,103.0,446,...,1014.0,6382.0,1.772778,2,2,Afternoon,0,38.0,not informed,2.0
197199,2015-02-18 02:39:35,2015-02-18 03:23:53,5913,1.0,4,4396,4,109.0,103.0,446,...,1180.0,2658.0,0.738333,2,2,Afternoon,0,38.0,barbecue,2.0
197200,2015-02-18 04:59:36,2015-02-18 05:26:05,5913,1.0,1,1798,1,49.0,52.0,446,...,1158.0,1589.0,0.441389,2,4,Afternoon,0,3.0,barbecue,2.0


In [43]:
x = whole_data_test.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_onshift_dashers','median'))
x.columns[:-1]

Index(['market_id', 'time_of_day'], dtype='object')

In [386]:
# display(whole_data_test[['total_onshift_dashers','availabe_dashers_onshift','total_outstanding_orders']].median())
print(f'''Overall median total_onshift_dashers = {whole_data_test[['total_onshift_dashers']].median().values[0]}''')
display(whole_data_test.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_onshift_dashers','median'),).pivot(index='market_id',columns='time_of_day', values='median_value')[['Morning', 'Afternoon', 'Evening', 'Night']])
print(f'''Overall median availabe_dashers_onshift = {whole_data_test[['availabe_dashers_onshift']].median().values[0]}''')
display(whole_data_test.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('availabe_dashers_onshift','median'),).pivot(index='market_id',columns='time_of_day', values='median_value')[['Morning', 'Afternoon', 'Evening', 'Night']])
# display(whole_data_test.groupby(['market_id','time_of_day'],as_index=False).agg(median_value = ('total_outstanding_orders','median'),).pivot(index='market_id',columns='time_of_day', values='median_value'))

Overall median total_onshift_dashers = 37.0


time_of_day,Morning,Afternoon,Evening,Night
market_id,,,,
1.0,5.0,24.0,15.0,14.0
2.0,20.0,76.0,39.0,33.0
3.0,5.0,19.0,10.0,11.0
4.0,11.0,72.0,47.0,30.0
5.0,9.0,24.0,15.0,14.0
6.0,2.0,16.5,48.0,27.5
not informed,7.0,43.0,25.0,24.0


Overall median availabe_dashers_onshift = 1.0


time_of_day,Morning,Afternoon,Evening,Night
market_id,,,,
1.0,0.0,0.0,0.0,0.0
2.0,0.0,4.0,1.0,1.0
3.0,0.0,0.0,0.0,1.0
4.0,1.0,4.0,1.0,3.0
5.0,2.0,3.0,2.0,2.0
6.0,0.0,1.0,0.0,0.0
not informed,1.0,1.0,0.0,0.0


In [374]:
whole_data_test['market_id'].value_counts()

market_id
2.0             54966
4.0             47628
1.0             37845
3.0             23212
5.0             18144
6.0             14498
not informed      909
Name: count, dtype: int64

In [ ]:
## Adding the trivial varialbes

#target

#temporal variables 


#
